In [194]:
import pandas as pd
import os
import sys 
import numpy as np
from functools import reduce
import itertools
from ast import literal_eval #converts object list to list of strings
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import statsmodels.api as sm
from statsmodels.formula.api import ols

# this points to a Python file with the function country_mappings (not used)
from combine_country_regions import country_mappings

# not great practice, but this removes warnings from the output
import warnings
warnings.filterwarnings("ignore")

# display settings so I can see more on the screen
desired_width=1000
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns',10)
pd.options.display.max_rows = 50


In [195]:

#############################################################
# set this to point to your folder or create a new folder,
# (in my case my computer is called jpark and I called the folder trade_warning) 
#############################################################
os.chdir(r'C:\Users\jpark\VisualStudio\Simpsons_BACI\\')
baci_data = r"C:\Users\jpark\Downloads\BACI_HS92_V202401b"


In [196]:

# points to country codes as defined by BACI
COUNTRY_CODES = baci_data + "\country_codes_V202401b.csv"
# point to product codes
PRODUCT_DESCRIPTION = baci_data + "\product_codes_HS22_V202401b.csv"
# add region data, might be better sources
ADD_REGIONS = r"baci_preparation\iso_countries_regions.csv"
# add short HS2 description (could be better descriptions)
SHORT_CODES = r"baci_preparation\hs6twodigits.csv"
# add long product description
LONG_DESCRIPTION = baci_data + "\product_codes_HS22_V202401b.csv"
# add gdp data
GDP_DATA = r"baci_preparation\global_gdp.csv"

class baci:
    '''baci class contains the methods to load baci data and add characteristics such as geographic and strategic'''
    def readindata(self, bacidata, verbose = False, tmp_save = True) -> pd.DataFrame:
        '''main method to read in baci data'''
        df1 = pd.read_csv(bacidata, usecols=['t','i','j','k','v','q'], 
                          dtype= {'t': 'int64',
                                  'i': 'int64', 
                                  'j': 'int64', 
                                  'k': 'object',
                                  'v': 'float64',
                                  'q': 'object'}
                          )

        # This is too complicated, but '   NA' should be converted to float
        df1['q'] = df1['q'].apply(lambda x: x.strip()) # remove spaces in data
        df1['q'].replace('NA', np.NaN, inplace=True)   # np.NaN is different than string NaN
        df1['q'] = df1['q'].astype(float)

        # rename columns to make them meaningful to humans
        df1.rename(columns={'t': 'Year', 'i': 'Exporter', 'j': 'Importer', 'k': 'Product', 'v': 'Value', 'q': 'Quantity'}, inplace=True)

        ROW_COUNT = df1.shape[0]
        print("ROW_COUNT ORIGINAL: ", ROW_COUNT)

        # replace number with name of country *exporter* 
        iso1 = pd.read_csv(COUNTRY_CODES, usecols=['country_code', 'country_iso3'])
        df1 = df1.merge(iso1, left_on="Exporter", right_on="country_code", how="left")
        df1.drop(columns=['country_code', 'Exporter'], inplace = True)
        df1.rename(columns={"country_iso3": "Exporter"}, inplace=True)
    
        # replace number with name of country *importer*
        df1 = df1.merge(iso1, left_on="Importer", right_on="country_code", how="left")
        df1.drop(columns=['country_code', 'Importer'], inplace = True)
        df1.rename(columns={"country_iso3": "Importer"}, inplace=True)

        # 2015 has some strange data, take only Values greater than 10.00, otherwise number of exporting countries in 2015 is an outlier
        df1 = df1[df1['Value'] > 0.00]

        # if verbose is True, this will print out
        if verbose:
            hcodes = [str(x)[0:2] for x in df1["Product"]]
            print(set(hcodes))
            print(len(set(hcodes)))

        # make product code and int, otherwise its an object which can be confusing
        df1['Product'] = df1['Product'].astype(int)    

        ROW_COUNT = df1.shape[0]
        print("ROW_COUNT ORIGINAL2: ", ROW_COUNT)

        return df1
    
    def addprodcode(self, data):
        '''add the product description if needed'''
        # add product_codes
        prodcodes = pd.read_csv(PRODUCT_DESCRIPTION, usecols=['code', 'description'])
        # product '9999AA' appears to be a filler--empty
        mask = prodcodes['code'] == '9999AA'
        prodcodes = prodcodes[~mask]
        # I love merges, note its a left merge, I want all baci data to have a code, but dont care for product codes without products.
        data = data.merge(prodcodes, left_on = "Product", right_on = "code", how = "left")
        
        ROW_COUNT = data.shape[0]
        print("ROW_COUNT addprodcode: ", ROW_COUNT)
        
        return data
    
    def addshortdescriptoProdname(self, data):
        '''Add short product description based on codes'''

        localdata = data.copy()

        # this is necessary because codes 1:9 should be 01:09
        prod_h6 = pd.read_csv(SHORT_CODES, dtype = str)

        # this is necessary because codes 1:9 should be 01:09
        prod_h6.loc[:, 'code'] = ["0" + x if len(x) == 1 else x for x in prod_h6['code'].astype(str)]

        # this is necessary because codes 1:9 should be 01:09
        localdata.loc[:, 'code'] = ["0" + x if len(x) == 5 else x for x in localdata['Product'].astype(str)]

        # get first two numbers
        localdata['shrtDescription'] = localdata['code'].astype(str).str[0:2]
        
        proddesc = localdata.merge(prod_h6, left_on="shrtDescription", right_on="code")
        
        proddesc['product'] = proddesc['product'] + "_" + proddesc['shrtDescription']
        
        proddesc.drop(columns = {'code_x', 'shrtDescription', 'code_y'}, inplace = True)

        proddesc.rename(columns = {"product": "code"}, inplace = True)

        ROW_COUNT = proddesc.shape[0]
        print("ROW_COUNT addshortdescriptoProdname: ", ROW_COUNT)

        return proddesc
    
    def addlongdescription(self, data):
        '''Add product product description based on codes'''
        localdata = data.copy()
        longdesc = pd.read_csv(LONG_DESCRIPTION, dtype = str)

        # this is necessary because codes 1:9 should be 01:09
        localdata.loc[:, 'Product'] = ["0" + x if len(x) == 5 else x for x in localdata['Product'].astype(str)]

        longdesc.rename(columns = {"code": "isocode"}, inplace=True)
        longproddesc = localdata.merge(longdesc, left_on="Product", right_on="isocode", how = 'left', suffixes = ['x', 'y'])
       
        r1 = localdata.shape[0]
        r2 = longproddesc.shape[0]
        assert r1 == r2

        ROW_COUNT = longproddesc.shape[0]
        print("ROW_COUNT addlongdescription: ", ROW_COUNT)

        return longproddesc
    
    def add_gdp(self, data, GDP, year):
        '''Join GDP to data'''

        ### join GDP to data
        
        # Exporters
        gdp = GDP[GDP.index == year]
        gdp = gdp.T
        gdp['Exporter_gdp'] = gdp.index
        
        gdp.rename(columns={year: year + "_gdp_Exporter"}, inplace=True)

        dataj = data.merge(gdp, left_on = "Exporter", right_on = "Exporter_gdp")
        dataj[year + '_gdp_Exporter'] = dataj[year + '_gdp_Exporter']/1e+6
        
        # Importers
        gdp = GDP[GDP.index == year]
        gdp = gdp.T
        gdp['Importer_gdp'] = gdp.index
        gdp.rename(columns={year: year + '_gdp_Importer'}, inplace=True)

        data = dataj.merge(gdp, left_on = "Importer", right_on = "Importer_gdp")
       
        data.drop(columns = ["Exporter_gdp", "Importer_gdp"], inplace=True)

        ROW_COUNT = data.shape[0]
        print("ROW_COUNT add_gdp: ", ROW_COUNT)

        return data
         

In [223]:

def GDPData():
    '''should alway be run, need to move to BACI class'''
    # https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?end=2022&start=1960&view=chart
    # Taiwan comes from IMF data, added by hand. https://www.imf.org/external/datamapper/NGDPD@WEO/OEMDC/ADVEC/WEOWORLD
    
    data = pd.read_csv(GDP_DATA, index_col=[1], skiprows=4)
    data = data.drop(columns=['Country Name', 'Indicator Code', 'Indicator Name'])
    data = data.T
    return data
GDP = GDPData()
GDP_sum = pd.DataFrame(GDP.sum(axis=1))
GDP_sum.index = GDP_sum.index.astype("int")

In [198]:

# #############################################################
# # INITIALIZE object, needs to be run to create a BACI object instance
bc1 = baci()
# #############################################################

ROW_COUNT ORIGINAL:  4981382
ROW_COUNT ORIGINAL2:  4981382
ROW_COUNT addshortdescriptoProdname:  4981382


,Year,Product,Value,Quantity,Exporter,Importer,Unnamed: 0,code
0,1995,841510,36.687,5.812,AFG,DZA,82,nuclear reactors boilers turbines_84
1,1995,570110,11.060,0.195,AFG,AND,56,carpets and other floor coverings_57
2,1995,80620,11.804,15.000,AFG,AUS,7,fruits_and_nuts_08
3,1995,570110,11.931,0.245,AFG,AUS,56,carpets and other floor coverings_57
4,1995,570210,3.692,0.377,AFG,AUS,56,carpets and other floor coverings_57
...,...,...,...,...,...,...,...,...
4981377,1995,940360,7.344,0.050,ZMB,USA,92,furniture bedding mattresses_94
4981378,1995,970500,36.072,NaN,ZMB,USA,95,works of art collectors pieces_97
4981379,1995,360300,13.508,0.566,ZMB,BFA,35,explosives pyrotechnic products_36
4981380,1995,491000,1.035,0.050,ZMB,BFA,48,printed books newspapers_49


In [200]:

years = np.arange(start=1995, stop=2023)

allYears = []

for yr in years:
    print(yr)
    bacidata = baci_data + "\BACI_HS92_Y" + str(yr) + "_V202401b.csv"
    test_data = bc1.readindata(bacidata, verbose = False, tmp_save = False)
    test_data = bc1.addshortdescriptoProdname(test_data)

    groupdata = test_data[['Value', 'code']].groupby('code').sum().T
    groupdata['Year'] = yr
    
    allYears.append(groupdata)

data1 = pd.concat(allYears, axis=0)
data1.set_index('Year', inplace=True)
data1.to_csv("tmp123.csv")
    
    
    

1995
ROW_COUNT ORIGINAL:  4981382
ROW_COUNT ORIGINAL2:  4981382
ROW_COUNT addshortdescriptoProdname:  4981382
1996
ROW_COUNT ORIGINAL:  5337010
ROW_COUNT ORIGINAL2:  5337010
ROW_COUNT addshortdescriptoProdname:  5337010
1997
ROW_COUNT ORIGINAL:  5616781
ROW_COUNT ORIGINAL2:  5616781
ROW_COUNT addshortdescriptoProdname:  5616781
1998
ROW_COUNT ORIGINAL:  5847753
ROW_COUNT ORIGINAL2:  5847753
ROW_COUNT addshortdescriptoProdname:  5847753
1999
ROW_COUNT ORIGINAL:  5994574
ROW_COUNT ORIGINAL2:  5994574
ROW_COUNT addshortdescriptoProdname:  5994574
2000
ROW_COUNT ORIGINAL:  7272734
ROW_COUNT ORIGINAL2:  7272734
ROW_COUNT addshortdescriptoProdname:  7272734
2001
ROW_COUNT ORIGINAL:  7517477
ROW_COUNT ORIGINAL2:  7517477
ROW_COUNT addshortdescriptoProdname:  7517477
2002
ROW_COUNT ORIGINAL:  7726196
ROW_COUNT ORIGINAL2:  7726196
ROW_COUNT addshortdescriptoProdname:  7726196
2003
ROW_COUNT ORIGINAL:  7981043
ROW_COUNT ORIGINAL2:  7981043
ROW_COUNT addshortdescriptoProdname:  7981043
2004
ROW_C

In [224]:
data1.merge(GDP_sum, left_index=True, right_index=True)

,aircraft spacecraft_88,albuminoidal substances_35,aluminium and articles thereof_76,animal_orginated_products_05,animal_vegetable_fats_15,...,wood and articles of wood_44,wool fine or coarse_51,works of art collectors pieces_97,zinc and articles thereof_79,0
1995,6.788237e+07,8.693783e+06,6.194809e+07,4.051236e+06,3.062783e+07,...,6.995624e+07,1.557253e+07,6.561453e+06,5.097880e+06,2.307468e+14
1996,7.552393e+07,9.210330e+06,5.871168e+07,4.245974e+06,2.953505e+07,...,7.098472e+07,1.518373e+07,6.422786e+06,5.211478e+06,2.381542e+14
1997,8.606628e+07,9.348892e+06,6.237430e+07,4.255718e+06,3.145668e+07,...,7.358429e+07,1.540856e+07,7.729994e+06,7.087211e+06,2.380886e+14
1998,1.106983e+08,9.774738e+06,6.311034e+07,4.152013e+06,3.317873e+07,...,6.611056e+07,1.277718e+07,8.085025e+06,6.321274e+06,2.378218e+14
1999,1.124921e+08,9.600234e+06,6.188064e+07,3.790889e+06,2.898761e+07,...,7.187825e+07,1.142173e+07,8.243764e+06,6.381087e+06,2.420476e+14
2000,1.130735e+08,1.001220e+07,6.956053e+07,3.980089e+06,2.452699e+07,...,7.210835e+07,1.242355e+07,1.008979e+07,6.983765e+06,2.503799e+14
2001,1.204398e+08,1.060027e+07,6.840142e+07,3.743160e+06,2.403610e+07,...,6.797059e+07,1.189449e+07,9.699231e+06,6.108655e+06,2.497043e+14
2002,1.184746e+08,1.118113e+07,7.039954e+07,3.973917e+06,2.993630e+07,...,7.333061e+07,1.142930e+07,9.628895e+06,5.828726e+06,2.596720e+14
2003,1.196348e+08,1.309460e+07,7.914610e+07,4.435524e+06,3.736264e+07,...,8.212163e+07,1.203944e+07,9.416440e+06,6.365470e+06,2.929229e+14
2004,1.247985e+08,1.502484e+07,9.863755e+07,5.143938e+06,4.407303e+07,...,1.006354e+08,1.357593e+07,1.192202e+07,7.944534e+06,3.329515e+14


In [221]:
GDP_sum.index

Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'], dtype='object')

In [214]:
data1.index

Index([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype='int32', name='Year')